<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Certamen 3 - Código Base</h2>
    <h2> <a href="#acknowledgements"> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.01</h2>
</center>

# No debe utilizar bibliotecas adicionales.

In [131]:
import numpy as np
import scipy as sp
# To solve IVP, notice this is different that odeint!
from scipy.integrate import solve_ivp
# To integrate use one of the followings:
from scipy.integrate import quad, quadrature, trapezoid, simpson
# For least-square problems
from scipy.linalg import lstsq, qr
# For interpolation
from scipy.interpolate import BarycentricInterpolator
# The wonderful GMRes
from scipy.sparse.linalg import gmres, LinearOperator
# The wonderful**2 Newton method coupled to GMRes by a matrix-free call!
from scipy.optimize import newton_krylov
from scipy.linalg import toeplitz

# Pregunta 1

In [132]:
# RHS of dynamical system
def my_f1(t,w): #sacado del jupyter del profe
    w1 = w[0]
    w2 = w[1]
    w1dot = w2
    #w2dot = (w1-1)*np.sqrt(pow((1+pow(w2,2)),3))
    w2dot = w1*pow(w2,2)*np.sqrt(pow(w2,2)+1)+w1*np.sqrt(pow(w2,2)+1)-pow(w2,2)*np.sqrt(pow(w2,2)+1)-np.sqrt(pow(w2,2)+1)
    return np.array([w1dot,w2dot])

# Function to be used to apply the Shooting Method
def F_SM_1(alpha,y0,y1,N):#sacado del jupyter del profe
    t = np.linspace(-1,1,N)
    initial_condition = np.zeros(2)
    initial_condition[0] = y0
    initial_condition[1] = alpha
    sol = solve_ivp(my_f1,(-1,1),initial_condition,t_eval=t)
    return sol.y[0,-1]-y1


'''
input:
N        : (int) número de puntos de discretización del intervalo [-1,1]

output:
x        : (ndarray) discretización del intervalo espacial [-1,1] en N puntos
y        : (ndarray) aproximación numérica de y(x) en los N puntos en que se discretizó 
            el intervalo espacial [-1,1]
'''
def solve_2nd_order_ODE(N):
    # Your own code
    F_root_1 = lambda alpha: F_SM_1(alpha,0,0,N)#sacado del jupyter del profe
    alpha_r = sp.optimize.root(F_root_1, 1.).x[0]#sacado del jupyter del profe
    sol = solve_ivp(my_f1,(-1,1),np.array([0,alpha_r]),t_eval=np.linspace(-1,1,N))#sacado del jupyter del profe
    y = sol.y[0,:]
    x = sol.t
    return x, y


'''
input:
N        : (int) número de puntos de discretización del intervalo [-1,1]

output:
integral : (double) aproximación numérica de la integral de y(x) entre -1 y 1.
'''
def compute_integral(N):
    # Your own code.
    x,y = solve_2nd_order_ODE(N)
    integral = trapezoid(y,x)
    return float(integral)

#print(compute_integral(100))

'''
input:
N        : (int) número de puntos de discretización del intervalo [-1,1]

output:
max_y    : (double) máximo valor de y(x) obtenido en la discretización espacial de N puntos de intervalo [-1,1],
            si existieran 2 máximos solo retornar el primero.
max_x    : (double) coordenada x donde se alcanza el máximo valor de y(x)
'''
def find_max(N):
    # Your own code.
    x,y = solve_2nd_order_ODE(N)
    max_y = 0
    max_x = 0
    for i in range(N):
        if(y[i]>max_y):
            max_y = y[i]
            max_x = x[i]
    return float(max_y), float(max_x)

#print(find_max(100))

'''
input:
m       : (double) pendiente objetivo
N       : (double) número de puntos de discretización del dominio [-1,1]
delta   : (double) umbral considerado para aceptar la aproximación de la pendiente m, 
            es decir |y(widehat_x)-m|<delta

output:
widehat_x      : (double) solución de la ecuación y(widehat_x)=m, considere -1<widehat_x<1
'''
def find_widehat_x(m, N, delta):
    # Your own code.    
    x,y = solve_2nd_order_ODE(N)
    h = abs(x[2]-x[1])
    widehat_x = np.nan
    for i in range(1,N-1):
        y_prima = (y[i+1]-y[i-1])/(2*h) #derivada con central difference
        sol = abs(y_prima - m)
        if (sol<delta):
            widehat_x = x[i]
            break #para en el primer valor que cumple la condición
    return float(widehat_x)

#find_widehat_x(0.0044, 100, 1)
#find_widehat_x(0.0044, 100, 1e-4)

# Pregunta 2

In [133]:
# This function computes the 'matrix-vector' product of the matrix we don't have explicitly stored!!
def compute_matrix_vector_product(x,A,B,n): #sacado de la función de sylvester del profesor
    X = np.reshape(x,(n,n),order='F')
    out = np.dot(A,X)+np.dot(X,B)
    return out.flatten('F')


'''
input:
A       : (ndarray) matriz input A
Q       : (ndarray) matriz input Q
Xm      : (ndarray) matriz input X^{(m)}
n       : (int) se considera que las dimensiones de las matrices son 'n x n'

output:
Xout         : (ndarray) matriz de n x n output X^{(m+1)}
rel_residual : (double)  residuo relativo del sistema de ecuaciones lineales.
'''
def obtain_next_X(A,Q,Xm,n):
    # Your own code.
    # This is part of the interface that SciPy requires.
    #reordenando la función queda como:
    #xm+1 * A_inv*xm - Q * A_inv * xm+1 = -A^T
    Q_2,R_2 = qr(A)
    Q_2 = Q_2.T
    A_inversa = sp.linalg.solve_triangular(R_2,Q_2)
    A_sylvester = -1*np.dot(Q,A_inversa)
    B_sylvester = np.dot(A_inversa,Xm)
    C_sylvester = -1*A.T
    Ax = lambda x: compute_matrix_vector_product(x,A_sylvester,B_sylvester,n)
    # This is the famous 'afun'!!
    afun = LinearOperator((n**2, n**2), matvec=Ax)
    # Just running GMRes
    x, exitCode = gmres(afun, C_sylvester.flatten('F'), tol=1e-10)
    # Just reshaping the 
    Xout = np.reshape(x,(n,n),order='F')
    rel_residual = np.linalg.norm(np.dot(A_sylvester,Xout)+np.dot(Xout,B_sylvester)-C_sylvester)
    return Xout, float(rel_residual)

A  = np.array([[10.5488135,   0.71518937], [0.60276338, 10.54488318]])
Q  = np.array([[2.37096596, 0.6638646 ], [0.6638646,  3.21243829]])
Xm = np.array([[1.0, 0.0 ], [0.0,  1.0]])
n = 2

#_,res = obtain_next_X(A,Q,Xm,n)
#print(np.log10(res))

In [134]:
# Particular test case.
n = 10 # Se  define 'n=10' solo a modo de ejemplo.
A  = np.eye(n)
Xm = np.eye(n)
Q  = 2*np.eye(n)

# Segundo caso
A = np.array([[10.5488135, 0.71518937], [0.60276338, 10.54488318]])
Q = np.array([[2.37096596, 0.6638646 ], [0.6638646, 3.21243829]])
Xm = np.array([[1.0, 0.0 ], [0.0, 1.0]])
n = 2